In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

# 그래프 한글처리
import matplotlib as mpl
# 한글 깨짐 방지
mpl.rcParams['font.family'].insert(0,'Malgun Gothic')

In [0]:
def rename_CD(x):
    if x == 10:
        return '숙박'
    elif x == 20:
        return '레저용품'
    elif x == 21:
        return '레저업소'
    elif x == 22:
        return '문화취미'
    elif x == 30:
        return '가구'
    elif x == 31:
        return '전기'
    elif x == 32:
        return '주방용구'
    elif x == 33:
        return '연료판매'
    elif x == 34:
        return '광학제품'
    elif x == 35:
        return'가전'
    elif x == 40:
        return '유통업'
    elif x == 42:
        return '의복'
    elif x == 43:
        return '직물'
    elif x == 44:
        return '신변잡화'
    elif x == 50:
        return '서적문구'
    elif x == 52:
        return '사무통신'
    elif x == 60:
        return '자동차판매'
    elif x == 62:
        return '자동차정비'
    elif x == 70:
        return '의료기관'
    elif x == 71:
        return '보건위생'
    elif x == 80:
        return '요식업소'
    elif x == 81:
        return '음료식품'
    elif x == 92:
        return '수리서비스'
    else:
        return x

In [4]:
sales = pd.read_csv('CARD_SPENDING.txt', sep='\t')

ParserError: ignored

In [0]:
sales.head()

In [0]:
sales['MCT_CAT'] = sales.MCT_CAT_CD.apply(rename_CD)

sales_jong = sales[sales.GU_CD == 110]
sales_no = sales[sales.GU_CD == 350]

In [0]:
sales.shape

In [0]:
sales.info()

In [0]:
sales.isnull().sum()

In [0]:
sales.GU_CD.value_counts()

# 350-노원구, 110-종로구

In [0]:
sales.STD_DD.min()

In [0]:
sales.STD_DD.max()

In [0]:
# 종로구 카드매출 데이터 
sales_jong = sales[sales.GU_CD == 110]
sales_jong.head()

In [0]:
# 노원구 카드매출 데이터 
sales_no = sales[sales.GU_CD == 350]

In [0]:
# YYYYmm 슬라이싱
# sales_jong['Ym'] = sales_jong.STD_DD.astype(str).str.slice(0,6)
# sales_jong.head()

### 1. 구별 업종별 이용 금액 비율, 이용 건수 비율

In [0]:
# 종로구
summ_jong = sales_jong.pivot_table(index='MCT_CAT',
                                   values=['USE_CNT', 'USE_AMT'],
                                   aggfunc='sum')

summ_jong['USE_AMT_PCT'] = round(summ_jong.USE_AMT / summ_jong.USE_AMT.sum() * 100, 2)
summ_jong['USE_CNT_PCT'] = round(summ_jong.USE_CNT / summ_jong.USE_CNT.sum() * 100, 2)

summ_jong = summ_jong.sort_values(by='USE_AMT_PCT', ascending=False)
summ_jong = summ_jong[:15]
summ_jong

In [0]:
# 노원구
summ_no = sales_no.pivot_table(index='MCT_CAT',
                               values=['USE_CNT', 'USE_AMT'],
                               aggfunc='sum')

summ_no['USE_AMT_PCT'] = round(summ_no.USE_AMT / summ_no.USE_AMT.sum() * 100, 2)
summ_no['USE_CNT_PCT'] = round(summ_no.USE_CNT / summ_no.USE_CNT.sum() * 100, 2)

summ_no = summ_no.sort_values(by='USE_AMT_PCT', ascending=False)
summ_no = summ_no[:15]

In [0]:
# 종로구

# Windonws 한글 폰트 설정
rc('font', family='NanumGothic')

x = summ_jong.index
y1 = summ_jong.USE_AMT_PCT
y2 = summ_jong.USE_CNT_PCT

ind = np.arange(15)
width = 0.35

plt.figure(figsize=(16,8))
plt.bar(ind - 0.2, y1, label='이용금액 비율', width=width, color='navy')
plt.bar(ind + 0.2, y2,label='이용건수 비율', width=width, color='blue')

plt.title('종로구 업종별 카드매출', fontsize=20)
plt.xticks(ind, x)
plt.legend()
plt.show()

In [0]:
# 노원구 

# Windonws 한글 폰트 설정
rc('font', family='NanumGothic')

x = summ_no.index
y1 = summ_no.USE_AMT_PCT
y2 = summ_no.USE_CNT_PCT

ind = np.arange(15)
width = 0.35

plt.figure(figsize=(16,8))
plt.bar(ind - 0.2, y1, label='이용금액 비율', width=width, color='darkgreen')
plt.bar(ind + 0.2, y2,label='이용건수 비율', width=width, color='seagreen')

plt.title('노원구 업종별 카드매출', fontsize=20)
plt.xticks(ind, x)
plt.legend()
plt.show()

### 2. 성별에 따른 이용 금액/건수에 차이가 있는지? 
- 이표본 t-test 검정
- 남자가 돈을 더 많이 쓰는지, 여자가 돈을 더 많이 쓰는지? 

In [0]:
sales_by_sex = sales_jong.pivot_table(index=['STD_DD', 'SEX_CD'], aggfunc='sum', values=['USE_CNT', 'USE_AMT'])
sales_by_M = sales_by_sex.xs('M', axis =0, level=1)
sales_by_F = sales_by_sex.xs('F', axis =0, level=1) 

In [0]:
sales_by_M.describe()

In [0]:
sales_by_F.describe()

In [0]:
from scipy import stats
tTestResult = stats.ttest_ind(sales_by_M.USE_AMT, sales_by_F.USE_AMT)

In [0]:
print('t검정통계량(등분산가정): %.3f, pvalue=%.3f'%(tTestResult))

In [0]:
tTestResultDiffVar = stats.ttest_ind(sales_by_M.USE_AMT, sales_by_F.USE_AMT, equal_var=False)

In [0]:
print('t검정통계량(등분산가정X): %.3f, pvalue=%.3f'%(tTestResultDiffVar))

### 3. 성별로 선호하는 업종이 있는지?

##### 종로구

In [0]:
sales_by_sex1 = sales_jong.pivot_table(index=['STD_DD', 'SEX_CD', 'MCT_CAT'], aggfunc='sum', values=['USE_CNT', 'USE_AMT'])
sales_by_F1 = sales_by_sex1.xs('F', axis =0, level=1)
sales_by_M1 = sales_by_sex1.xs('M', axis =0, level=1)

In [0]:
# 여성
sales_by_F1 = sales_by_F1.pivot_table(index='MCT_CAT', aggfunc='sum',
                                      values=['USE_AMT', 'USE_CNT']).sort_values(by='USE_AMT', ascending=False)

sales_by_F1['USE_AMT_PCT'] = round(sales_by_F1.USE_AMT / sales_by_F1.USE_AMT.sum() * 100, 2)
sales_by_F1['USE_CNT_PCT'] = round(sales_by_F1.USE_CNT / sales_by_F1.USE_CNT.sum() * 100, 2)

sales_by_F1 = sales_by_F1.sort_values(by='USE_AMT_PCT', ascending=False)

sales_by_F1 = sales_by_F1[:15]

In [0]:
# 여성

# Windonws 한글 폰트 설정
rc('font', family='NanumGothic')

x = sales_by_F1.index
y1 = sales_by_F1.USE_AMT_PCT
y2 = sales_by_F1.USE_CNT_PCT

ind = np.arange(len(sales_by_F1))
width = 0.35

plt.figure(figsize=(16,8))
plt.bar(ind - 0.2, y1, label='이용금액 비율', width=width, color='orange')
plt.bar(ind + 0.2, y2,label='이용건수 비율', width=width, color='gold')

plt.title('종로구 여성 업종별 카드매출', fontsize=20)
plt.xticks(ind, x)
plt.legend()
plt.show()

In [0]:
# 남성
sales_by_M1 = sales_by_M1.pivot_table(index='MCT_CAT', aggfunc='sum', 
                                      values=['USE_AMT', 'USE_CNT']).sort_values(by='USE_AMT', ascending=False)

sales_by_M1['USE_AMT_PCT'] = round(sales_by_M1.USE_AMT / sales_by_M1.USE_AMT.sum() * 100, 2)
sales_by_M1['USE_CNT_PCT'] = round(sales_by_M1.USE_CNT / sales_by_M1.USE_CNT.sum() * 100, 2)

sales_by_M1 = sales_by_M1.sort_values(by='USE_AMT_PCT', ascending=False)

sales_by_M1 = sales_by_M1[:15]

In [0]:
## 남성

# Windonws 한글 폰트 설정
rc('font', family='NanumGothic')

x = sales_by_M1.index
y1 = sales_by_M1.USE_AMT_PCT
y2 = sales_by_M1.USE_CNT_PCT

ind = np.arange(len(sales_by_M1))
width = 0.35

plt.figure(figsize=(16,8))
plt.bar(ind - 0.2, y1, label='이용금액 비율', width=width, color='darkmagenta')
plt.bar(ind + 0.2, y2,label='이용건수 비율', width=width, color='orchid')

plt.title('종로구 남성 업종별 카드매출', fontsize=20)
plt.xticks(ind, x)
plt.legend()
plt.show()

##### 노원구

In [0]:
sales_by_sex2 = sales_no.pivot_table(index=['STD_DD', 'SEX_CD', 'MCT_CAT'], aggfunc='sum', values=['USE_CNT', 'USE_AMT'])
sales_by_F2 = sales_by_sex2.xs('F', axis =0, level=1)
sales_by_M2 = sales_by_sex2.xs('M', axis =0, level=1)

In [0]:
# 여성
sales_by_F2 = sales_by_F2.pivot_table(index='MCT_CAT', aggfunc='sum',
                                      values=['USE_AMT', 'USE_CNT']).sort_values(by='USE_AMT', ascending=False)

sales_by_F2['USE_AMT_PCT'] = round(sales_by_F2.USE_AMT / sales_by_F2.USE_AMT.sum() * 100, 2)
sales_by_F2['USE_CNT_PCT'] = round(sales_by_F2.USE_CNT / sales_by_F2.USE_CNT.sum() * 100, 2)

sales_by_F2 = sales_by_F2.sort_values(by='USE_AMT_PCT', ascending=False)

sales_by_F2 = sales_by_F2[:15]

# Windonws 한글 폰트 설정
rc('font', family='NanumGothic')

x = sales_by_F2.index
y1 = sales_by_F2.USE_AMT_PCT
y2 = sales_by_F2.USE_CNT_PCT

ind = np.arange(len(sales_by_F2))
width = 0.35

plt.figure(figsize=(16,8))
plt.bar(ind - 0.2, y1, label='이용금액 비율', width=width, color='orange')
plt.bar(ind + 0.2, y2,label='이용건수 비율', width=width, color='gold')

plt.title('노원구 여성 업종별 카드매출', fontsize=20)
plt.xticks(ind, x)
plt.legend()
plt.show()

In [0]:
# 남성
sales_by_M2 = sales_by_M2.pivot_table(index='MCT_CAT', aggfunc='sum',
                                      values=['USE_AMT', 'USE_CNT']).sort_values(by='USE_AMT', ascending=False)

sales_by_M2['USE_AMT_PCT'] = round(sales_by_M2.USE_AMT / sales_by_M2.USE_AMT.sum() * 100, 2)
sales_by_M2['USE_CNT_PCT'] = round(sales_by_M2.USE_CNT / sales_by_M2.USE_CNT.sum() * 100, 2)

sales_by_M2 = sales_by_M2.sort_values(by='USE_AMT_PCT', ascending=False)

sales_by_M2 = sales_by_M2[:15]

# Windonws 한글 폰트 설정
rc('font', family='NanumGothic')

x = sales_by_M2.index
y1 = sales_by_M2.USE_AMT_PCT
y2 = sales_by_M2.USE_CNT_PCT

ind = np.arange(len(sales_by_M2))
width = 0.35

plt.figure(figsize=(16,8))
plt.bar(ind - 0.2, y1, label='이용금액 비율', width=width, color='darkmagenta')
plt.bar(ind + 0.2, y2,label='이용건수 비율', width=width, color='orchid')

plt.title('노원구 남성 업종별 카드매출', fontsize=20)
plt.xticks(ind, x)
plt.legend()
plt.show()

### 4. 나이대별 선호하는 업종에 차이가 있을까?

1.   항목 추가
2.   항목 추가



In [0]:
sales.head()

In [0]:
sales.AGE_CD.value_counts()

In [0]:
sales_by_age = sales.pivot_table(index=['MCT_CAT', 'AGE_CD'], aggfunc='sum', values=['USE_CNT', 'USE_AMT'])
sales_by_age.xs(20, axis=0, level=1)
sales_by_age.xs(25, axis=0, level=1)
sales_by_age.xs(30, axis=0, level=1)
sales_by_age.xs(35, axis=0, level=1)
sales_by_age.xs(40, axis=0, level=1)
sales_by_age.xs(45, axis=0, level=1)
sales_by_age.xs(50, axis=0, level=1)
sales_by_age.xs(55, axis=0, level=1)
sales_by_age.xs(60, axis=0, level=1)
sales_by_age.xs(65, axis=0, level=1)


In [0]:
ages = [20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
sales_by_ages = {}

In [0]:
for age in ages:
    sales_by_ages[age] = sales_by_age.xs(age, axis=0, level=1)

    sales_by_ages[age]['USE_AMT_PCT'] = round(sales_by_ages[age].USE_AMT / sales_by_ages[age].USE_AMT.sum() * 100, 2)
    sales_by_ages[age]['USE_CNT_PCT'] = round(sales_by_ages[age].USE_CNT / sales_by_ages[age].USE_CNT.sum() * 100, 2)

    sales_by_ages[age] = sales_by_ages[age].sort_values(by='USE_AMT_PCT', ascending=False)

    sales_by_ages[age] = sales_by_ages[age][:15]

    # Windonws 한글 폰트 설정
    rc('font', family='NanumGothic')

    x = sales_by_ages[age].index
    y1 = sales_by_ages[age].USE_AMT_PCT
    y2 = sales_by_ages[age].USE_CNT_PCT

    ind = np.arange(len(sales_by_ages[age]))
    width = 0.35

    plt.figure(figsize=(16,8))
    plt.bar(ind - 0.2, y1, label='이용금액 비율', width=width, color='olive')
    plt.bar(ind + 0.2, y2,label='이용건수 비율', width=width, color='y')

    plt.title('{0}대 업종별 카드매출'.format(age), fontsize=20)
    plt.xticks(ind, x)
    plt.legend()
    plt.show()